In [ ]:
%%capture
!pip install langchain openai datasets

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# FewShotPromptTemplate

The `FewShotPromptTemplate` allows you to provide a few examples to prime the model before your main prompt. This can be useful for providing relevant context and "warming up" the model on your task.

You may want to use a `FewShotPromptTemplate` when:

 - Your task requires some background context to perform well. For example, summarization tasks often benefit from example summaries.

 - You want to bias the model towards a certain style or tone. Example outputs in that style can help.

> FewShotPromptTemplate allows providing sample I/O to prime the model for your task. Use it when you need extra context, style priming, or have very short prompts.

### To use FewShotPromptTemplate:

1. Create the template, providing a list of example dicts with "input" and "output" keys. Each example should be a dictionary with the keys being the input variables and the values being the values for those input variables.

2. Create a formatter for the few shot examples

3. Feed examples and formatter to `FewShotPromptTemplate`

This will return a prompt with your examples, then the main input.

### The key aspects are:

-  Examples that demonstrate the desired input/output format

- Examples that provide relevant context

- The main prompt is formatted to match the style of the examples

This allows priming the model in a more nuanced way than just providing a generic instruction like "Answer the following geography questions."

The `FewShotPromptTemplate` allows assembling a prompt that guides the model using targeted examples, rather than just high-level instructions. This results in a more nuanced prompt that better primes the model for the specific task.

In [ ]:
from langchain.prompts import FewShotPromptTemplate

# Examples that demonstrate the desired input/output format
examples = [
    {
        "question": "Who rocked the fashion scene better, Cleopatra or Queen Elizabeth I?",
        "answer": (
            "Is a fashion runway showdown needed here: Totally!\n"
            "Follow up: Describe Cleopatra's iconic style.\n"
            "Intermediate answer: Cleopatra was known for her kohl-lined eyes and intricate jewelry.\n"
            "Follow up: And Queen Elizabeth I's fashion statement?\n"
            "Intermediate answer: Elizabeth I was all about the ruffs, pearls, and powdered wigs.\n"
            "Final Verdict: Cleopatra, with her exotic allure, narrowly takes the crown!"
        )
    },
    {
        "question": "When did the creator of Myspace join the internet party?",
        "answer": (
            "Ready for a digital time travel? Let's dive!\n"
            "Follow up: Who started Myspace?\n"
            "Intermediate answer: Myspace was co-founded by Chris DeWolfe and Tom Anderson.\n"
            "Follow up: When did they launch their masterpiece?\n"
            "Intermediate answer: Myspace was introduced to the world in 2003.\n"
            "Final Verdict: 2003 - When Myspace became the internet's coolest hangout."
        )
    },
    {
        "question": "Who was the great-grandfather of Winston Churchill?",
        "answer": (
            "Going down the ancestry rabbit hole, are we?\n"
            "Follow up: Who was Winston's father?\n"
            "Intermediate answer: Winston Churchill's father was Lord Randolph Churchill.\n"
            "Follow up: And who was Lord Randolph's grandpa?\n"
            "Intermediate answer: That would be George Spencer-Churchill, the 5th Duke of Marlborough.\n"
            "Final Verdict: George Spencer-Churchill is the answer, no doubt about it!"
        )
    },
    {
        "question": "Do the creators of 'Mario' and 'Sonic' hail from the same land?",
        "answer": (
            "Ready for a gaming face-off?\n"
            "Follow up: Who's the genius behind 'Mario'?\n"
            "Intermediate Answer: 'Mario' was crafted by Shigeru Miyamoto of Nintendo.\n"
            "Follow up: And from where does Miyamoto hail?\n"
            "Intermediate Answer: Japan, the land of sushi and sumo!\n"
            "Follow up: And the mastermind behind 'Sonic'?\n"
            "Intermediate Answer: 'Sonic' was created by Yuji Naka for Sega.\n"
            "Follow up: And Yuji Naka's home country?\n"
            "Intermediate Answer: Also Japan, where gaming legends are born!\n"
            "Final Verdict: Undoubtedly, both legends emerged from Japan."
        )
    }
]

In [ ]:
from langchain.prompts.prompt import PromptTemplate
# Create a formatter for the few-shot examples

example_prompt = PromptTemplate(input_variables=["question", "answer"],
                                template="Question: {question}\n{answer}")

In [ ]:
# Feed examples and formatter to FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="Who had better taste Steve Jobs or Bill Gates?"))

Question: Who rocked the fashion scene better, Cleopatra or Queen Elizabeth I?
Is a fashion runway showdown needed here: Totally!
Follow up: Describe Cleopatra's iconic style.
Intermediate answer: Cleopatra was known for her kohl-lined eyes and intricate jewelry.
Follow up: And Queen Elizabeth I's fashion statement?
Intermediate answer: Elizabeth I was all about the ruffs, pearls, and powdered wigs.
Final Verdict: Cleopatra, with her exotic allure, narrowly takes the crown!

Question: When did the creator of Myspace join the internet party?
Ready for a digital time travel? Let's dive!
Follow up: Who started Myspace?
Intermediate answer: Myspace was co-founded by Chris DeWolfe and Tom Anderson.
Follow up: When did they launch their masterpiece?
Intermediate answer: Myspace was introduced to the world in 2003.
Final Verdict: 2003 - When Myspace became the internet's coolest hangout.

Question: Who was the great-grandfather of Winston Churchill?
Going down the ancestry rabbit hole, are we

In [ ]:
# Pass this to an llm

from langchain.llms import OpenAI

llm = OpenAI()

print(llm(prompt.format(input="Who had better fashion taste Steve Jobs or Bill Gates?")))


Time to size up the style titans!
Follow up: What was Steve Jobs' style?
Intermediate Answer: Steve Jobs was known for his iconic black turtlenecks and jeans.
Follow up: And Bill Gates' fashion statement?
Intermediate Answer: Bill Gates was often seen in khakis and plain-colored sweaters.
Final Verdict: Steve Jobs' iconic style wins out in this fashion battle!


# Let's engineer this prompt

As language models like LLMs continue to expand their context capacity, it's becoming more common to incorporate numerous class examples in prompts.

However, as the diversity and number of classes increase, manually curating examples for every class becomes a challenging task.

This often results in a reliance on random selection from a collection of examples.

We'll use this [dataset](https://huggingface.co/datasets/Bhuvaneshwari/intent_classification)

In [ ]:
%%capture
from datasets import load_dataset
dataset = load_dataset("Bhuvaneshwari/intent_classification", split='train')

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
intent_dataset = dataset.to_pandas()
intent_dataset.head()

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist


In [ ]:
import random
from typing import List, Dict, Union
import pandas as pd

def get_prefix(data: Union[List[Dict[str, str]], 'pd.DataFrame'], prefix: str = "") -> str:
    """
    Generate a prefix string for the prompt to guide the language model in intent selection.

    The function takes a dataset (which can be a list of dictionaries or a pandas DataFrame)
    with 'text' and 'intent' columns, and an optional prefix string. It returns a formatted string
    that lists all unique intents from the dataset, prefixed with the provided prefix.

    Parameters:
    - data: List or a pandas DataFrame containing text and its associated intent.
    - prefix: Optional string to be prepended to the result.

    Returns:
    - str: A formatted string listing all unique intents from the dataset.
    """
    # Extract unique intents from the dataset
    if isinstance(data, list):
        unique_intents = set(item['intent'] for item in data)
    else:
        unique_intents = data.intent.unique()

    # Format the prefix, list of intents, and other parts of the string
    prompt = f"{prefix}\n" if prefix else ""
    prompt += f"Choose from the following intents: {', '.join(unique_intents)}"

    return prompt

In [ ]:
print(get_prefix(intent_dataset, "Each input is associated with a user intent."))

Each input belongs is associated with a user intent
Choose from the following intents: PlayMusic, AddToPlaylist, RateBook, Greetings, SearchScreeningEvent, BookRestaurant, GetWeather, Book Meeting, SearchCreativeWork, Cancellation, Affirmation, excitment


In [ ]:
def get_examples(data: Union[List[Dict[str, str]], 'pd.DataFrame']) -> List[Dict[str, str]]:
    """
    Extract one random example from each unique intent present in the dataset.

    The function processes a dataset (which can be a list of dictionaries or a pandas DataFrame)
    with 'text' and 'intent' columns. It returns a list containing one random example for each
    unique intent in the dataset.

    Parameters:
    - data: Dataset containing text and its associated intent.

    Returns:
    - List[Dict[str, str]]: A list of dictionaries, each containing a 'text' and 'intent' pair.
    """

    # Set a seed for reproducibility
    random.seed(42)

    # Group the data by 'intent', then take a random sample from each group
    samples = data.groupby('intent').apply(lambda group: group.sample(n=1)).reset_index(drop=True)

    # Convert the sampled data to a list of dictionaries
    examples = samples.to_dict(orient='records')

    return examples

# Using the function to get examples from the dataset
examples_dataset = get_examples(intent_dataset)
examples_dataset

[{'text': 'add anthony hamilton to my list women of hip hop',
  'intent': 'AddToPlaylist'},
 {'text': 'indeed', 'intent': 'Affirmation'},
 {'text': 'Please schedule a meeting with Kumar at 5 PM and sent invite to email ID',
  'intent': 'Book Meeting'},
 {'text': 'book a brasserie in the falkland islands for this week',
  'intent': 'BookRestaurant'},
 {'text': 'I’m declining', 'intent': 'Cancellation'},
 {'text': 'what will the temperature be at midnight in ne',
  'intent': 'GetWeather'},
 {'text': 'Good Afternoon!', 'intent': 'Greetings'},
 {'text': 'play the top five songs from robert lockwood junior',
  'intent': 'PlayMusic'},
 {'text': 'rate this book three out of 6', 'intent': 'RateBook'},
 {'text': 'i need a photograph of the oregon desert',
  'intent': 'SearchCreativeWork'},
 {'text': 'show me movie schedules', 'intent': 'SearchScreeningEvent'},
 {'text': 'Wow', 'intent': 'excitment'}]

In [ ]:
from typing import List, Dict, Union

def construct_prompt(data: Union[List[Dict[str, str]], 'pd.DataFrame'],
               text: str,
               examples: List[Dict[str, str]],
               prefix: str = "") -> str:
    """
    Construct a formatted k-shot prompt using:
    - A prefix.
    - Specified examples (one from each intent).
    - Target text for classification.

    The function takes a dataset, a target text, examples, and an optional prefix. It then uses these
    to craft a structured k-shot prompt using the PromptTemplate and FewShotPromptTemplate classes.

    Parameters:
    - data: Dataset containing text and its associated intent.
    - text: Target text to be classified.
    - examples: List of dictionaries, each containing a 'text' and 'intent' pair.
    - prefix: Optional string to be prepended to the prompt.

    Returns:
    - str: Formatted k-shot prompt.
    """

    # Define the prompt template with specified format
    prompt_template = PromptTemplate(
        input_variables=["text", "intent"],
        template="Text: {text}\nLabel: {intent}",
    )

    # Construct the few-shot prompt using the provided examples, prefix, and other configurations
    prompt = FewShotPromptTemplate(
        example_prompt = prompt_template,
        examples = examples,
        prefix = get_prefix(data, prefix),
        suffix = "Text: {text}\nLabel:",
        input_variables = ['text'],
    )

    # Return the formatted prompt
    return prompt.format(text=text).strip()

# Example usage

prompt = construct_prompt(intent_dataset,
                 "Save Free Smoke by AP Dhillon to my songs",
                 examples_dataset,
                 "Each input belongs is associated with a user intent")

print(prompt)

Each input belongs is associated with a user intent
Choose from the following intents: PlayMusic, AddToPlaylist, RateBook, Greetings, SearchScreeningEvent, BookRestaurant, GetWeather, Book Meeting, SearchCreativeWork, Cancellation, Affirmation, excitment

Text: add anthony hamilton to my list women of hip hop
Label: AddToPlaylist

Text: indeed
Label: Affirmation

Text: Please schedule a meeting with Kumar at 5 PM and sent invite to email ID
Label: Book Meeting

Text: book a brasserie in the falkland islands for this week
Label: BookRestaurant

Text: I’m declining
Label: Cancellation

Text: what will the temperature be at midnight in ne
Label: GetWeather

Text: Good Afternoon!
Label: Greetings

Text: play the top five songs from robert lockwood junior
Label: PlayMusic

Text: rate this book three out of 6
Label: RateBook

Text: i need a photograph of the oregon desert
Label: SearchCreativeWork

Text: show me movie schedules
Label: SearchScreeningEvent

Text: Wow
Label: excitment

Text: S

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI()

llm(prompt)

' AddToPlaylist'